In [1]:
import json
from pprint import pprint
import pandas as pd
import numpy as np

In [2]:
data = []
with open('/home/julia/Загрузки/Telegram Desktop/552670aa4eae356bf4d0d11c.jsonl') as raw_data:
    for line in raw_data:
        data.append(json.loads(line))
#pprint(data)
df = pd.DataFrame(data)
df['created'] = pd.to_datetime(df['created'])
df =  df.sort_values('created' )
#df['created'] = df.created.astype(str)
df['word'] = df.word.str.join(',')
print(type(df.get_value(3,'title')))
df['day'] = df.created.dt.day
df['hour'] = df.created.dt.hour
df['minute'] = df.created.dt.minute
df['date'] = df['day'].map(str) + "-" + df['hour'].map(str) + "-" + df['minute'].map(str)
#z = (np.unique(dff[['word']].values))
#z2 = len(sorted(set((",".join(z).split(",")))))
df

<class 'str'>


,created,language,source,text,title,url,word,day,hour,minute,date
100,2017-09-15 11:01:44,ru,Транспортный бизнес,"44 часа понадобилось ГП ""МТП ""Южный"" для разгр...","В ГП ""МТП ""Южный"" балкер с углем из США разгру...",http://tbu.com.ua/news/v_gp_mtp_ujnyi_balker_s...,"мтп,центрэнерг,южн,суток,сша,углем,ambitious,д...",15,11,1,15-11-1
264,2017-09-15 11:03:09,ru,2000.ua,"Статья 5 закона Украины ""О повышении престижно...","""Шатун"" высшего образования",https://www.2000.ua/v-nomere/derzhava/realii/s...,"закон,вуз,стипенд,детям,шахтер,студент,повышен...",15,11,3,15-11-3
266,2017-09-15 11:03:36,uk,ЕраМедіа,"Згідно з чинним законодавством, органи державн...","Кличко пояснив, чому блокпакет Київенерго опин...",http://www.eramedia.com.ua/article/268572-klic...,"акц,кличк,пакет,влад,державн,блокуюч,вплив,киї...",15,11,3,15-11-3
99,2017-09-15 11:04:23,ru,ОстроВ,Подача воды с Красноармейской фильтровальной с...,Ряд городов Донетчины останется без воды из-за...,http://www.ostro.org/donetsk/society/news/532768/,"вод,красноармейск,карловк,станц,подач,город,фи...",15,11,4,15-11-4
265,2017-09-15 11:05:11,ru,2000.ua,Счета за тепло- и водоснабжение потребителям б...,Украинцы будут платить за дырявые трубы,https://www.2000.ua/novosti/ekonomika_novosti/...,"учет,услуг,закон,коммерческ,коммуналн,тепл,общ...",15,11,5,15-11-5
97,2017-09-15 11:13:05,ru,06239.com.ua Покровск и Мирноград,"В среду, 20 сентября, с 8:00 в связи с плановы...",20 сентября Покровск и Мирноград на сутки оста...,https://www.06239.com.ua/news/1795567,"вод,игор,карловк,красноармейск,планов,юдин,рем...",15,11,13,15-11-13
98,2017-09-15 11:13:13,uk,Insider,"Згідно з чинним законодавством, органи державн...","Кличко пояснив, чому блокпакет ""Київенерго"" оп...",http://www.theinsider.ua/politics/59bbb221762a7/,"київенерг,акц,державн,кличк,влад,пакет,законод...",15,11,13,15-11-13
96,2017-09-15 11:13:52,ru,ПрессОрг24,Соломенский районный суд Киева удовлетворил хо...,"Суд дал НАБУ доступ к счетам по делу ""Роттердам+""",http://pressorg24.com/news?id=357204,"платеж,решен,суд,доступ,счетам,принят,банковск...",15,11,13,15-11-13
249,2017-09-15 11:14:22,ru,ИнформБюро,В Донецкой Народной Республике завершено форми...,Под эгидой РП РЭК объединили 5 предприятий с в...,http://informburo.dn.ua/cgi-bin/iburo/start.cg...,"предприят,дтэк,управлен,ооо,рэк,днр,завершен,р...",15,11,14,15-11-14
250,2017-09-15 11:14:56,ru,Репортер.dn.ua,"Пять украинских энергетических предприятий, на...",В ДНР пять национализированных украинских пред...,http://reporter.dn.ua/news/economy/v_dnr_pyat_...,"дтэк,предприят,ооо,днр,рэк,угл,украинск,управл...",15,11,14,15-11-14


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=5, max_features=10000, ngram_range=(1, 2))
vz = vectorizer.fit_transform(df['word'])
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
tfidf = pd.DataFrame(columns=['tfidf']).from_dict(dict(tfidf), orient='index')
tfidf.columns = ['tfidf']
#tfidf.sort_values(by=['tfidf'], ascending=True).head(30)

In [13]:
from sklearn.cluster import MiniBatchKMeans

num_clusters = 35
kmeans_model = MiniBatchKMeans(n_clusters=num_clusters, init='k-means++', n_init=1, 
                         init_size=1000, batch_size=1000, verbose=False, max_iter=1000)
kmeans = kmeans_model.fit(vz)
kmeans_clusters = kmeans.predict(vz)
kmeans_distances = kmeans.transform(vz)

In [14]:
for i, desc in enumerate(df.title):
    if(i < 295):
        if kmeans_clusters[i] == 9:
            print("Cluster " + str(kmeans_clusters[i]) + ": " + desc + 
                  "(distance: " + str(kmeans_distances[i][kmeans_clusters[i]]) + ")")
            print('---')

Cluster 9: ДТЭК Энерго направит более 8 млрд грн в развитие производства в 2017 году(distance: 0.585353472101)
---
Cluster 9: ДТЭК Энерго направит более 8 млрд грн в развитие производства в 2017 году(distance: 0.481536051394)
---
Cluster 9: ДТЭК "Энерго" направит более 8 млрд грн на развитие производства в 2017 году(distance: 0.496107169184)
---
Cluster 9: "ДТЭК Энерго" в I полугодии увеличило капинвестиции на 64%(distance: 0.650871992504)
---
Cluster 9: ДТЭК Энерго направит более 8 млрд грн в развитие производства в 2017 году(distance: 0.565932602695)
---
Cluster 9: "ДТЭК Энерго" в I полугодии увеличило капинвестиции на 64%(distance: 0.650871992504)
---
Cluster 9: ДТЭК Энерго направит более 8 млрд грн в развитие производства в 2017 году(distance: 0.582836926458)
---
Cluster 9: (distance: 0.680567382736)
---
Cluster 9: На что энергокомпания Ахметова потратила 3,6 млрд гривень(distance: 0.770903866538)
---
Cluster 9: Компания Ахметова инвестировала более 3 млрд грн в добычу угля(distanc

In [5]:
import lda
from sklearn.feature_extraction.text import CountVectorizer


In [6]:
import logging
logging.getLogger("lda").setLevel(logging.WARNING)

In [7]:
cvectorizer = CountVectorizer(min_df=4, max_features=10000, ngram_range=(1,2))
cvz = cvectorizer.fit_transform(df['word'])

n_topics = 32
n_iter = 2000
lda_model = lda.LDA(n_topics=n_topics, n_iter=n_iter)
X_topics = lda_model.fit_transform(cvz)

In [30]:
from sklearn.manifold import TSNE
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=100, random_state=0,algorithm='arpack')
svd_tfidf = svd.fit_transform(vz)
tsne_model = TSNE(n_components=2, verbose=1, random_state=0)
tsne_tfidf = tsne_model.fit_transform(svd_tfidf)
#tsne_lda = tsne_model.fit_transform(X_topics)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 295 samples in 0.001s...
[t-SNE] Computed neighbors for 295 samples in 0.014s...
[t-SNE] Computed conditional probabilities for sample 295 / 295
[t-SNE] Mean sigma: 0.281648
[t-SNE] KL divergence after 250 iterations with early exaggeration: 66.317497
[t-SNE] Error after 1000 iterations: 0.454729


In [31]:
doc_topic = lda_model.doc_topic_
lda_keys = []
for i, tweet in enumerate(df['text']):
    lda_keys += [doc_topic[i].argmax()]

In [32]:
import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook
plot_lda = bp.figure(plot_width=700, plot_height=600, title="LDA topic visualization",
    tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
    x_axis_type=None, y_axis_type=None, min_border=1)

In [38]:
lda_df = pd.DataFrame(tsne_lda, columns=['x','y'])
lda_df['text'] = df['text']
lda_df['title'] = df['title']


In [39]:
tsne_lda = tsne_model.fit_transform(X_topics)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 295 samples in 0.001s...
[t-SNE] Computed neighbors for 295 samples in 0.015s...
[t-SNE] Computed conditional probabilities for sample 295 / 295
[t-SNE] Mean sigma: 0.281648
[t-SNE] KL divergence after 250 iterations with early exaggeration: 66.317497
[t-SNE] Error after 1000 iterations: 0.454729


In [40]:
lda_df['topic'] = lda_keys
lda_df['topic'] = lda_df['topic'].map(int)
colormap = np.array(["#6d8dca", "#69de53", "#723bca", "#c3e14c", "#c84dc9", "#68af4e", "#6e6cd5",
"#e3be38", "#4e2d7c", "#5fdfa8", "#d34690", "#3f6d31", "#d44427", "#7fcdd8", "#cb4053", "#5e9981",
"#803a62", "#9b9e39", "#c88cca", "#e1c37b", "#34223b", "#bdd8a3", "#6e3326", "#cfbdce", "#d07d3c",
"#52697d", "#7d6d33", "#d27c88", "#36422b", "#b68f79", "#7fff00", "#ffa500", "#ff1493", "#fac205", 
                     "#40fd14","#FF0000","#FF0000","#FF0000","#FF0000","#FF0000","#FF0000","#FF0000"
                    ,"#FF0000","#FF0000","#FF0000","#FF0000","#FF0000"])

In [41]:
plot_lda.scatter(source=lda_df, x='x', y='y', color=colormap[lda_keys])

hover = plot_lda.select(dict(type=HoverTool))
hover.tooltips={"text":"@title", "topic":"@title"}
show(plot_lda)

/usr/local/lib/python3.5/dist-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
/usr/local/lib/python3.5/dist-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
